In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train_data.shape

(891, 12)

In [4]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
X_test = test_data

In [5]:
y = train_data.Survived
X = train_data.drop(['Survived'], axis = 1)

In [6]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked']

X = X[features]
X_test = X_test[features]

In [7]:
# Get names of columns with missing values
cols_with_missing = [col for col in X.columns
                     if X[col].isnull().any()]
print(cols_with_missing)

['Age', 'Embarked']


In [8]:
from sklearn.impute import SimpleImputer

numeric_columns = X.select_dtypes(include='number').columns
num_Imputer = SimpleImputer(strategy='median')

X[numeric_columns] = pd.DataFrame(num_Imputer.fit_transform(X[numeric_columns]), columns=numeric_columns)
X_test[numeric_columns] = pd.DataFrame(num_Imputer.fit_transform(X_test[numeric_columns]), columns=numeric_columns)

/tmp/ipykernel_18/3207151343.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[numeric_columns] = pd.DataFrame(num_Imputer.fit_transform(X_test[numeric_columns]), columns=numeric_columns)


In [9]:
cat_columns = X.select_dtypes(include='object').columns
cat_Imputer = SimpleImputer(strategy='most_frequent')

X[cat_columns] = pd.DataFrame(cat_Imputer.fit_transform(X[cat_columns]), columns=cat_columns)
X_test[cat_columns] = pd.DataFrame(cat_Imputer.fit_transform(X_test[cat_columns]), columns=cat_columns)

/tmp/ipykernel_18/3306425986.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[cat_columns] = pd.DataFrame(cat_Imputer.fit_transform(X_test[cat_columns]), columns=cat_columns)


In [10]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()

X[cat_columns] = ordinal_encoder.fit_transform(X[cat_columns])
X_test[cat_columns] = ordinal_encoder.transform(X_test[cat_columns])

/tmp/ipykernel_18/3104000814.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[cat_columns] = ordinal_encoder.transform(X_test[cat_columns])


In [11]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth=5, max_features=0.5, max_samples=0.8, n_estimators=125)

In [12]:
from sklearn.model_selection import RandomizedSearchCV as RSCV

param_grid = {'n_estimators':np.arange(50,200,15),
              'max_features':np.arange(0.1, 1, 0.1),
              'max_depth': [3, 5, 7, 9],
              'max_samples': [0.3, 0.5, 0.8]}

model = RSCV(RandomForestClassifier(), param_grid, n_iter = 15).fit(X, y)
model = model.best_estimator_

print(model)

RandomForestClassifier(max_depth=5, max_features=0.8, max_samples=0.5,
                       n_estimators=110)


In [13]:
from sklearn.metrics import mean_absolute_error

model.fit(X, y)

# view the feature scores

feature_scores = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)

feature_scores

Sex         0.505442
Pclass      0.187003
Age         0.174616
SibSp       0.060596
Embarked    0.039290
Parch       0.033053
dtype: float64

In [14]:
preds = model.predict(X_test)

In [15]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': preds})
output.to_csv('submission.csv', index=False)